In [1]:
!pip install pythainlp

In [2]:
import deepcut
import pandas as pd
import numpy as np
import re
from itertools import chain
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from flask import Flask, jsonify, request

## data from all category

In [3]:
from itertools import chain
def tokenize_text_list(ls):
    """Tokenize list of text"""
    return list(chain.from_iterable([deepcut.tokenize(ls)]))

In [4]:
# ลงทะเบียนเรียน
enrollment = ['ชำระเงินค่าลงทะเบียนแล้ว แต่ในระบบยังขึ้นว่าไม่ชำระเงิน',
  'ถ้าต้องการลงทะเบียนเรียนน้อยกว่า 9 หน่วยกิตต้องทำอย่างไร?',
  'ลากิจต้องทำอย่างไร?',
  'ผมอยากลาออกต้องทำยังไง?',
  'ใบแจ้งชำระเงินค่าลงทะเบียน สั่งพิมพ์ได้จากที่ไหน',
  'แจ้งขอสำเร็จการศึกษาได้ที่ไหน',
  'ถ้าวิชาเรียน sec เต็มแล้วทำยังไง',
  'ถ้าจะจบการศึกษา แต่เรียน gen ซ้ำหมวดกันทำยังไง',
  'ลงทะเบียนเรียนล่าช้าใช้เอกสารอะไรบ้าง',
  'หากมีเหตุต้องพักการเรียน สามารถพักการเรียนตอนนี้ได้ไหม ถ้าได้ต้องทำอย่างไรบ้าง?',
  'ดรอปได้ถึงเมื่อไหร่?',
  'สามารถเช็คกำหนดการลงทะเบียนเรียนได้ที่ไหน?',
  'ต้องการสอบโดยไม่เข้าเรียนได้ไหม?',
  'ยังต้องเข้าไปเรียนที่มหาลัยไหม?',
  'ลืมจ่ายเงินค่าลงทะเบียน และ ใบแจ้งหนี้เกินกำหนดไปแล้ว ทำยังไง?',
  'ลาป่วยต้องทำอย่างไร? ',
  'ตารางสอบซ้อนต้องทำยังไง?',
  'ฉันสามารถลงวิชาเสริมในเทอมนี้ได้มากที่สุดกี่ตัว',
  'ถ้าจะต้องการเอาเครื่องคิดเลขเข้้าห้องสอบต้องทำยังไง',
  'ถ้าต้องการลงทะเบียนเรียนมากกว่า 19 หน่วยกิตต้องทำอย่างไร?',
  'กรณีเพิ่มและหลักสูตรปิดรายวิชา จะได้รับเงินคืนไหม หรือ ต้องจ่ายเงินเพิ่มเท่าไหร่',
  'ลงทะเบียนเรียนเพิ่มรายวิชา ชำระเงินได้ที่ไหน?',
  'อยากได้เอกสารเบิกค่าเทอมต้องเตรียมอะไรบ้าง',
  'ตอนดรอปต้องขอลายเซ็นใครบ้าง?',
  'ตรวจวันเวลาสอบได้ที่ไหน',
  'สามารถลงทะเบียนเกินหน่วยกิตที่หลักสูตรกำหนดได้หรือไม่',
  'เรียนต่อต่างประเทศต้องใช้เอกสารอะไรบ้าง',
  'หากจบการศึกษาไม่ทันเกณฑ์ทหารทำยังไง',
  'ลงทะเบียนล่าช้าหลังระบบลงทะเบียนปิดที่ไหน',
  'ถ้าการย้ายหลักสูตรต้องทำอย่างไร?',
  'กรณีเพิ่มและลดรายวิชา จะได้รับเงินคืนไหม หรือ ต้องจ่ายเงินเพิ่มเท่าไหร่',
  'สามารถเช็คคอร์สเรียนออนไลน์ได้ที่ไหน',
  'ถ้าจะดึงบางวิชาลงมาเรียนแทนตัวที่ติด f ได้ไหม ต้องทำอย่างไร',
  'วิชาเลือกไหนน่าสนใจ?',
  'ขอทรานสคริปต์ออนไลน์ได้ที่ไหน',
  'ถ้าต้องการเปลี่ยน section ต้องทำอย่างไร',
  'ขอลดรายวิชาออนไลน์ หลังหมดเขต เพิ่ม-ลด 2 สัปดาห์แรกได้ที่ไหน?']

In [5]:
# การรับเข้านักศึกษา
admission = ['วิศวะคอม/ TCAS รอบ 2 มีโครงการไหนเปิดรับบ้าง?',
  'ถ้าจะสมัครเข้าโครงการ Active Recruitment วัดผลยังไง? ดูอะไรบ้าง?',
  'วิศวะคอม/ TCAS รอบ 4 ใช้คะแนนสอบอะไรบ้าง?',
  'หลักสูตรปกติมีเกณฑ์การรับยังไงบ้าง?',
  'หลักสูตรHDSมีเกณฑ์การรับยังไงบ้าง?',
  'สมัครเข้าโครงการ Active Recruitment ได้อย่างไร?',
  'วิศวะคอม/ TCAS รอบ 3ใช้คะแนนสอบอะไรบ้าง?',
  'หลักสูตรนานาชาติมีเกณฑ์การรับยังไงบ้าง?',
  'ถ้าสนใจจะเรียนวิศวะคอม/ เปิดรับเมื่อไหร่ ช่วงไหนบ้าง?',
  'วิศวะคอม/ สามารถสมัครเข้าเรียนยังไงได้บ้าง มีรอบอะไรเปิดรับบ้าง?',
  'แต่ละหลักสูตรมีเกณฑ์การรับพื้นฐานยังไงบ้าง?',
  'ถ้าจะสมัครเข้าโครงการ Active Recruitment ใช้เอกสารอะไรบ้าง?',
  'วิศวะคอม/ TCAS รอบ 1 มีโครงการไหนเปิดรับบ้าง?',
  'การับสมัครเด็กเข้าเรียน มีทั้งหมดกี่รอบ? อะไรบ้าง?',
  'สามารถติดต่อข่าวสารช่องทางการรับสมัครได้ที่ช่องทางไหนบ้าง? ',
  'จะต้องใช้เอกสารอะไรบ้างในการสมัครเข้าศึกษา']

In [6]:
# ฝึกงาน
internship = ['ถ้าไม่ได้เป็นนักศึกษาชั้นปีที่ 3 สามารถฝึกงานได้หรือไม่？',
  'หากไม่สามารถหาที่ฝึกงานในช่วงนี้ได้ จะต้องดรอปวิชาฝึกงานไหม',
  'Work Integrate Leaning หรือ Wil จำเป็นที่ลงเลือกวิชาเลือกเสรีหรือไม่?',
  'หากไม่่สามารถฝึกงานได้ต้องทำอย่างไร',
  'ถ้าต้องการฝึกงานต้องทำอย่างไรบ้าง?',
  'ฝึกงานจำเป็นต้องครบ40ชั่วโมงไหม？',
  'Work Integrate Leaning หรือ Wil ทำตั้งแต่ช่วงเวลาไหนถึงไหน?',
  'ถ้าต้องการทำ Work Integrate Leaning หรือ Wil ต้องทำอย่างไร?']

In [7]:
#หลักสูตร
curriculum = ['วิชา CPE332/Professional issuesมีวิชาตัวต่อมั้ยครับ?',
  'วิชาเลือกที่มีวิชาบังคับลงก่อน หลักสูตรปกติ?',
  'วิศวะคอม/หลักสูตรนานาชาติ ปี 4 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 4 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาเลือกที่มีวิชาบังคับลงก่อน หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ ?',
  'วิชา CPE375/Interactive computingมีวิชาตัวต่อมั้ยครับ?',
  'หน่วยกิตที่ต้องเก็บให้ครบก่อนจบหรือไม่?',
  'วิศวะคอม/วิศวกรรมคอมพิวเตอหลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 3 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE224/Computer architectures มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรนานาชาติปี 1 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติปี 1 เทอม 1 \nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาเลือกที่มีวิชาบังคับลงก่อน หลักสูตรนานาชาติ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ มีกี่หน่วยกิต? ',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ ปีที่1-2 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติปี 2 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE212/Algorithm มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรปกติ  ปี 4 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอมหลักสูตรวิทยาศาตร์ข้อมูลสุขภาพมีอัตราค่าเรียนเท่าไหร่',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 3 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาภาคบังคับที่มีวิชาต่อเนื่อง มีอะไรบ้าง หลักสูตรปกติ?',
  'วิชา CPE314/Computer networksมีวิชาตัวต่อมั้ยครับ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรเรสิเดนทอล คอเลจ ปีที่ 1-2 เรียนอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ ปีที่ 3 เรียนอะไรบ้าง?',
  'วิชา CPE327/Software engineering มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอมพิวเตอร์แต่ละหลักสูตรมีระยะเวลาในการศึกษากี่ปี?',
  'วิชา CPE325/Big data มีวิชาตัวต่อมั้ยครับ?',
  'วิชา CPE111/Data Structure มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรปกติ ปี 2 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติ ปี 2 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาเลือกเสรีต้องลงวิชานอกภาคเท่านั้นหรือไม่?',
  'วิศวะคอมหลักสูตรปกติมีอัตราค่าเรียนเท่าไหร่',
  'วิชา CPE326/Operating systems มีวิชาตัวต่อมั้ยครับ?',
  'วิชา CPE121/Discrete มีวิชาตัวต่อมั้ยครับ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ ปีที่ 4 เรียนอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ มีกี่หน่วยกิต?',
  'วิชา CPE342/Java programmingมีวิชาตัวต่อมั้ยครับ?',
  'วิชาภาษาอังกฤษ LNG มีเกณท์การเรื่มเรียนอย่างไร หลักสูตรวิทยาศาสตรบัณฑิต\nสาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ ?',
  'วิชา CPE343/Object orientedมีวิชาตัวต่อมั้ยครับ?',
  'วิชา CPE100/Programming มีวิชาตัวต่อมั้ยครับ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ มีกี่หน่วยกิต?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 1 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรปกติ ปี 3 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่1-2 เรียนอะไรบ้าง?',
  'วิชา CPE223/Digital มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรนานาชาติปี 4 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE101/Exploration มีวิชาตัวต่อมั้ยครับ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรเรสิเดนทอล คอเลจ ปีที่ 3-4 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติ ปี 3 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาภาษาอังกฤษ LNG มีเกณท์การเรื่มเรียนอย่างไร หลักสูตรปกติ ?',
  'วิชาภาคบังคับที่มีตัวต่อเนื่องมีอะไรบ้าง หลักสูตรนานาชาติ?',
  'วิศวะคอม/หลักสูตรปกติ  ปี 3 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ ปีที่1-2 เรียนอะไรบ้าง?',
  'วิชา CPE231/Database มีวิชาตัวต่อมั้ยครับ?',
  'วิชาภาคบังคับที่มีวิชาต่อเนื่อง  หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nมีอะไรบ้าง ?',
  'วิชาภาษาอังกฤษ LNG มีเกณท์การเรื่มเรียนอย่างไร หลักสูตรนานาชาติ ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ ปีที่ 3 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรปกติ  ปี 2 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรปกติ ปี 1 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอมหลักสูตรปกติมีระยะเวลาในการศึกษากี่ปี?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 2 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE213/Data model วิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 1 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/ มีทั้งหมดกี่หลักสูตร? มีหลักสูตรอะไรบ้าง?',
  'วิศวะคอมหลักสูตรนานาชาติมีอัตราค่าเรียนเท่าไหร่',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 4 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่ 4 เรียนอะไรบ้าง?',
  'วิชา CPE122/Circuits มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรปกติ  ปี 4 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่ 3 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติ ปี 3 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE329/Business intelligence มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 2 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ ปีที่ 4 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรปกติ ปี 1 เทอม 1 \nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?']

In [8]:
# คำถามทั่วไป
faq = ['เวลาต้องการยื่นเอกสารต้องยื่นที่ไหน',
  'วิศวะคอม/หลักสูตรเรสิเดนทอล คอลเลจจบแล้วได้ปริญญาอะไร?',
  'ขั้นตอนและช่วงเวลาเปิดรับสมัครนักศึกษา?',
  'หลักสูตรวิทยาศาสตร์ข้อมูลสุขภาพจบมาแล้วทำงานในสายใดบ้าง?',
  'วิศวะคอม/หลักสูตรปกติจบแล้วได้ปริญญาอะไร?',
  'เรียนจบต้องใช้คะแนนอังกฤษอะไรบ้าง ขั้นตํ่าเท่าไหร่??',
  'หลักสูตรนานาชาติจบมาแล้วทำงานในสายใดบ้าง?',
  'สามารถติดต่ออ.ได้ช่องทางไหนบ้าง?',
  'สมัครหลักสูตรยังไง?',
  'ขั้นตอนการรับสมัครโครงการทุนการศึกษา?',
  'วิศวะคอม/เรียนเกี่ยวกับอะไร?',
  '/วิศวะคอมพิวเตอร์ เรียนจบทำงานอะไร',
  'การส่งคะแนนสอบภาษาอังกฤษเพื่อสำเร็จการศึกษา ส่งอย่างไร?',
  'ใครเป็นหัวหน้าภาควิชาวิศวะคอม/?',
  'มีที่ฝึกงานแนะนำไหม?',
  'ติดต่อวิศวะคอม/ได้ทางช่องทางไหนได้บ้าง?',
  'สามารถติดต่อได้ทางช่องทางไหนบ้าง ช่วงเวลากี่โมงถึงกี่โมง?',
  'หลักสูตรเรสิเดรเทียล คอเจน์จบมาแล้วทำงานในสายใดบ้าง?',
  'วิศวะคอม/แตกต่างจาก IT อย่างไร?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตร์ข้อมูลสุขภาพจบแล้วได้ปริญญาอะไร?',
  'เกียรตินิยมอันดับ 2 อยู่ที่เท่าไหร่?',
  'หลักสูตรวิิทยาศาสตร์ข้อมูลสุขภาพคืออะไร?',
  'วิศวะคอม/จบไปทำงานอะไร?',
  'อาจารย์ในภาควิชาวิศวะคอม/มีใครบ้าง? ',
  'เกียรตินิยมอันดับ 1 อยู่ที่เท่าไหร่?',
  'หลักสูตรปกติจบมาแล้วทำงานในสายใดบ้าง?',
  'วิศวะคอม/แล้วได้ปริญญาอะไร?',
  'วิศวะคอม/หลักสูตรนานาชาติจบแล้วได้ปริญญาอะไร?',
  'วิศวะคอม/อยู่ที่ไหน?']

In [9]:
# ทุนการศึกษา
scholarship = ['สามารถขอทุนอะไรบ้าง?',
  'ทุนเพชรพระจอมเกล้าภาควิชารับกี่คน？',
  'ทุนของภาควิชามีอะไรบ้าง？',
  'การสมัครกู้ยืมเรียน ทำอย่างไร?',
  'ถ้าอยากไปเรียนต่อต่างและขอทุนต่างประเมศต้องทำอย่างไรบ้าง？',
  'มีเงินไม่พอจ่าย ทำอะไรได้บ้าง?',
  'ทุนจ้างงานคืออะไร？',
  'ปัจจุบันมีทุนอะไรเปิดรับบ้าง?']

In [10]:
enrollmentObj = {'enrollment':enrollment}
admissionObj = {'admission':admission}
internshipObj = {'internship':internship}
curriculumObj = {'curriculum':curriculum}
faqObj = {'faq':faq}
scholarshipObj = {'scholarship':scholarship}

In [11]:
enrollmentDF = pd.DataFrame(data=enrollmentObj)
admissionDF = pd.DataFrame(data=admissionObj)
internshipDF = pd.DataFrame(data=internshipObj)
curriculumDF = pd.DataFrame(data=curriculumObj)
faqDF = pd.DataFrame(data=faqObj)
scholarshipDF = pd.DataFrame(data=scholarshipObj)

In [12]:
beforeTok = {}
beforeTok['ลงทะเบียนเรียน'] = enrollmentDF
beforeTok['การรับเข้านักศึกษา'] = admissionDF
beforeTok['ฝึกงาน'] = internshipDF
beforeTok['หลักสูตร'] = curriculumDF
beforeTok['คำถามทั่วไป'] = faqDF
beforeTok['ทุนการศึกษา'] = scholarshipDF

In [13]:
beforeTok

{'ลงทะเบียนเรียน':                                            enrollment
 0   ชำระเงินค่าลงทะเบียนแล้ว แต่ในระบบยังขึ้นว่าไม...
 1   ถ้าต้องการลงทะเบียนเรียนน้อยกว่า 9 หน่วยกิตต้อ...
 2                                 ลากิจต้องทำอย่างไร?
 3                             ผมอยากลาออกต้องทำยังไง?
 4    ใบแจ้งชำระเงินค่าลงทะเบียน สั่งพิมพ์ได้จากที่ไหน
 5                       แจ้งขอสำเร็จการศึกษาได้ที่ไหน
 6                    ถ้าวิชาเรียน sec เต็มแล้วทำยังไง
 7      ถ้าจะจบการศึกษา แต่เรียน gen ซ้ำหมวดกันทำยังไง
 8               ลงทะเบียนเรียนล่าช้าใช้เอกสารอะไรบ้าง
 9   หากมีเหตุต้องพักการเรียน สามารถพักการเรียนตอนน...
 10                               ดรอปได้ถึงเมื่อไหร่?
 11         สามารถเช็คกำหนดการลงทะเบียนเรียนได้ที่ไหน?
 12                   ต้องการสอบโดยไม่เข้าเรียนได้ไหม?
 13                    ยังต้องเข้าไปเรียนที่มหาลัยไหม?
 14  ลืมจ่ายเงินค่าลงทะเบียน และ ใบแจ้งหนี้เกินกำหน...
 15                              ลาป่วยต้องทำอย่างไร? 
 16                           ตารางสอบซ้อนต้องท

In [14]:
tokenized_enrollment =enrollmentDF.enrollment.map(tokenize_text_list)
tokenized_admission =admissionDF.admission.map(tokenize_text_list)
tokenized_internship =internshipDF.internship.map(tokenize_text_list)
tokenized_curriculum =curriculumDF.curriculum.map(tokenize_text_list)
tokenized_faq =faqDF.faq.map(tokenize_text_list)
tokenized_scholarship =scholarshipDF.scholarship.map(tokenize_text_list)

In [15]:
tokenized = {}
tokenized['ลงทะเบียนเรียน'] = tokenized_enrollment
tokenized['การรับเข้านักศึกษา'] = tokenized_admission
tokenized['ฝึกงาน'] = tokenized_internship
tokenized['หลักสูตร'] = tokenized_curriculum
tokenized['คำถามทั่วไป'] = tokenized_faq
tokenized['ทุนการศึกษา'] = tokenized_scholarship

In [16]:
tokenized_len = {}
tokenized_len['ลงทะเบียนเรียน'] = len(tokenized['ลงทะเบียนเรียน'])
tokenized_len['การรับเข้านักศึกษา'] = len(tokenized['การรับเข้านักศึกษา'])
tokenized_len['ฝึกงาน'] = len(tokenized['ฝึกงาน'])
tokenized_len['หลักสูตร'] = len(tokenized['หลักสูตร'])
tokenized_len['คำถามทั่วไป'] = len(tokenized['คำถามทั่วไป'])
tokenized_len['ทุนการศึกษา'] = len(tokenized['ทุนการศึกษา'])

## category

In [17]:
data = pd.read_excel("Category.xlsx")

In [18]:
#Lib
import pickle
#Load File
with open('token_text_category.data', 'rb') as filehandle:
    # read the data as binary data stream
    tokenized_texts = pickle.load(filehandle)

In [19]:
def text_to_bow(tokenized_text, vocabulary_):
    n_doc = len(tokenized_text)
    values, row_indices, col_indices = [], [], []
    for r, tokens in enumerate(tokenized_text):
        feature = {}
        for token in tokens:
            word_index = vocabulary_.get(token)
            if word_index is not None:
                if word_index not in feature.keys():
                    feature[word_index] = 1
                else:
                    feature[word_index] += 1
        for c, v in feature.items():
            values.append(v)
            row_indices.append(r)
            col_indices.append(c)
        #print(feature)

    # document-term matrix in sparse CSR format
    X = sp.csr_matrix((values, (row_indices, col_indices)),
                      shape=(n_doc, len(vocabulary_)))
    return X

vocabulary_ = {v: k for k, v in enumerate(set(chain.from_iterable(tokenized_texts)))}
X = text_to_bow(tokenized_texts, vocabulary_)

In [20]:
transformer = TfidfTransformer()
svd_model = TruncatedSVD(n_components=100,
                         algorithm='arpack', n_iter=100)
X_tfidf = transformer.fit_transform(X)
X_svd = svd_model.fit_transform(X_tfidf)

In [21]:
tag = pd.get_dummies(data.Category).columns

In [22]:
import joblib
#Load Model
logist_models = joblib.load("category_model.pkl")

In [23]:
y_pred = np.argmax(np.vstack([model.predict_proba(X_svd)[:, 1] for model in logist_models]).T, axis=1)
y_pred = np.array([tag[yi] for yi in y_pred])
y_true = data.Category.values
print(tag[0:4])

Index(['การรับเข้านักศึกษา', 'คำถามทั่วไป', 'ลงทะเบียนเรียน', 'หลักสูตร'], dtype='object')


In [24]:
#Clean Text
def remove_repettition(text):
    token_list = list(text)
    if len(token_list) > 2:
        filter_list = [True, True]
        n = len(token_list)
        for i in range(2, n):
            if (token_list[i] == token_list[i-1]) and (token_list[i] == token_list[i-2]):
                filter_list.append(False)
            else:
                filter_list.append(True)

        output = ''.join(np.array(token_list)[filter_list])
    else:
        output = text
    return output

def cleansing(text):
    # \t, \n, \xa0 and other special characters. Replace by blank string
    text = re.sub('[\t\n\xa0\"\'!?\/\(\)%\:\=\-\+\*\_ๆ]', '', text)
    
    # Numbers. Replace by space
    text = re.sub('[0-9]', ' ', text)
    
    # Dot. Replace by space
    text = re.sub('[\.]', ' ', text)
    
    # One or more consecutive space. Replace by single space
    text = re.sub('\s+',' ',text)
    
    # Remove 2 or more repettition
    text = remove_repettition(text)
    
    return text

In [25]:
textInput = 'วิศวคอมพิวเตอร์มีหลักสูตรอะไรบ้าง'
textInput= cleansing(textInput)
tokenized_text = deepcut.tokenize(textInput)
x = text_to_bow([tokenized_text], vocabulary_)
x_tfidf = transformer.transform(x)
x_svd = svd_model.transform(x_tfidf)
pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
print(list(zip(tag, pred)))
predict_category = max(list(zip(tag, pred)))

[('การรับเข้านักศึกษา', 0.15501360454034896), ('คำถามทั่วไป', 0.23211256402801883), ('ลงทะเบียนเรียน', 0.0801201977246547), ('หลักสูตร', 0.48394961588403484)]


In [26]:
max_value = 0
max_category = ''
pred_results = list(zip(tag, pred))
for pred_result in pred_results:
  # print(pred_result)
  if pred_result[1] > max_value:
    max_value = pred_result[1]
    max_category = pred_result[0]
print(max_category, max_value)

หลักสูตร 0.48394961588403484


## prediction

In [27]:
#cosine
from pythainlp import word_tokenize 
from pythainlp.word_vector import * 
from sklearn.metrics.pairwise import cosine_similarity  
import numpy as np
model=get_model()
def sentence_vectorizer(ss,dim=300,use_mean=True): 
    s = word_tokenize(ss)
    vec = np.zeros((1,dim))
    for word in s:
        if word in model.wv.index2word:
            vec+= model.wv.word_vec(word)
        else: pass
    if use_mean: vec /= len(s)
    return vec
def sentence_similarity(s1,s2):
    return cosine_similarity(sentence_vectorizer(str(s1)),sentence_vectorizer(str(s2)))

C:\Users\NATTHAWATTUNGRUETHAI\AppData\Roaming\Python\Python38\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [28]:
len(beforeTok[max_category])

74

In [29]:
word = beforeTok[max_category].iloc[0]

In [30]:
word[0]

'วิชา CPE332/Professional issuesมีวิชาตัวต่อมั้ยครับ?'

In [31]:
word

curriculum    วิชา CPE332/Professional issuesมีวิชาตัวต่อมั้...
Name: 0, dtype: object

In [32]:
import json

In [33]:
result = []
for index in range(len(beforeTok[max_category])):
#     print(beforeTok[max_category].iloc[index])
    word = beforeTok[max_category].iloc[index]
#     print(word[0])
#     word = beforeTok[max_category].iloc[index]
    subResult = sentence_similarity(textInput,word[0])
    subResult = subResult[0][0]
    result.append(subResult)

<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.w

In [34]:
max_question_percentage = max(result)
print(max_question_percentage)
question_index = np.where(result == max_question_percentage)
print(question_index)
predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
predictedQuestion = predictedQuestion[0]

0.7615871650925202
(array([49], dtype=int64),)


In [35]:
predictedQuestion

'วิชาภาคบังคับที่มีตัวต่อเนื่องมีอะไรบ้าง หลักสูตรนานาชาติ?'

# time total

In [36]:
%%time
textInput = "วิศวคอมมีหลักสูตรอะไรบ้าง"
textInput= cleansing(textInput)
tokenized_text = deepcut.tokenize(textInput)
x = text_to_bow([tokenized_text], vocabulary_)
x_tfidf = transformer.transform(x)
x_svd = svd_model.transform(x_tfidf)
pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
print(list(zip(tag, pred)))
predict_category = max(list(zip(tag, pred)))

max_value = 0
max_category = ''
pred_results = list(zip(tag, pred))
for pred_result in pred_results:
  # print(pred_result)
    if pred_result[1] > max_value:
        max_value = pred_result[1]
        max_category = pred_result[0]
print(max_category, max_value)

result = []
for index in range(len(beforeTok[max_category])):
    word = beforeTok[max_category].iloc[index]
    subResult = sentence_similarity(textInput,word[0])
    subResult = subResult[0][0]
    result.append(subResult)
max_question_percentage = max(result)
print(max_question_percentage)
question_index = np.where(result == max_question_percentage)
print(question_index)
#         print(beforeTok[max_category].loc[question_index[0][0]])
predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
predictedQuestion = predictedQuestion[0]
print(predictedQuestion)
value = {
    "category": max_category,
    "accuracy": "%lf" % max_value,
    "predictedQuestion": str(predictedQuestion),
    "similarity": "%lf" % max_question_percentage
}

[('การรับเข้านักศึกษา', 0.15429524542160913), ('คำถามทั่วไป', 0.28921758989289553), ('ลงทะเบียนเรียน', 0.07409858686178913), ('หลักสูตร', 0.43034208776352584)]
หลักสูตร 0.43034208776352584


<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

0.7717235367264967
(array([62], dtype=int64),)
วิศวะคอม/ มีทั้งหมดกี่หลักสูตร? มีหลักสูตรอะไรบ้าง?
Wall time: 698 ms


<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

# time seperate

In [46]:
%%time
textInput = "วิศวคอมมีหลักสูตรอะไรบ้าง"
textInput= cleansing(textInput)

Wall time: 0 ns


In [47]:
%%time
tokenized_text = deepcut.tokenize(textInput)

Wall time: 57.8 ms


In [48]:
%%time
x = text_to_bow([tokenized_text], vocabulary_)
x_tfidf = transformer.transform(x)
x_svd = svd_model.transform(x_tfidf)

Wall time: 2 ms


In [49]:
%%time
pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
# print(list(zip(tag, pred)))

Wall time: 1.02 ms


In [50]:
%%time
predict_category = max(list(zip(tag, pred)))
max_value = 0
max_category = ''
pred_results = list(zip(tag, pred))

Wall time: 0 ns


In [51]:
%%time
for pred_result in pred_results:
  # print(pred_result)
    if pred_result[1] > max_value:
        max_value = pred_result[1]
        max_category = pred_result[0]
print(max_category, max_value)
# end of category model

หลักสูตร 0.43034208776352584
Wall time: 0 ns


In [52]:
%%time
result = []
for index in range(len(beforeTok[max_category])):
    word = beforeTok[max_category].iloc[index]
    subResult = sentence_similarity(textInput,word[0])
    subResult = subResult[0][0]
    result.append(subResult)

<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

Wall time: 629 ms


<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-27-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-27-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.w

In [53]:
%%time
max_question_percentage = max(result)
# print(max_question_percentage)
question_index = np.where(result == max_question_percentage)
# print(question_index)
# print(beforeTok[max_category].loc[question_index[0][0]])
predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
predictedQuestion = predictedQuestion[0]
# print(predictedQuestion)

Wall time: 0 ns


In [54]:
%%time
value = {
    "category": max_category,
    "accuracy": "%lf" % max_value,
    "predictedQuestion": str(predictedQuestion),
    "similarity": "%lf" % max_question_percentage
}

Wall time: 0 ns
